In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, vstack, hstack
from sklearn.preprocessing import MaxAbsScaler
from implicit.nearest_neighbours import TFIDFRecommender, ItemItemRecommender, CosineRecommender
from time import time
pd.set_option('display.max_columns', None)

from tqdm.notebook import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

# TRAIN SUBMIT

In [2]:
# Начало работы кода - для непрерывного запуска
start = time()

In [3]:
%%time
player_starts = pd.read_parquet('../input/player_starts_train.parquet')
player_starts

CPU times: user 24 s, sys: 4.4 s, total: 28.4 s
Wall time: 22.3 s


,date,user_id,item_id,watch_time,is_autorized
0,2023-07-21 19:04:50+03:00,user_12964323,video_1042531,51,0
1,2023-07-21 02:02:41+03:00,user_16517,video_1707159,31,0
2,2023-07-21 22:00:47+03:00,user_15057892,video_1989987,9,0
3,2023-07-21 19:09:43+03:00,user_2846972,video_1356486,-1,0
4,2023-07-21 11:06:58+03:00,user_20517034,video_1380654,11,0
...,...,...,...,...,...
69954375,2023-08-21 02:51:53+03:00,user_15478739,video_1449287,291,1
69954376,2023-08-21 08:40:18+03:00,user_25783543,video_1423321,2,0
69954377,2023-08-21 05:19:55+03:00,user_3507470,video_464555,261,0
69954378,2023-08-21 12:32:32+03:00,user_13128840,video_420973,21,0


In [4]:
sample_submission = pd.read_csv('../input/sample_submission.csv', sep=',', index_col=None)
sample_submission

,user_id,recs
0,user_26511551,"['video_0', 'video_0', 'video_0', 'video_0', '..."
1,user_29194819,"['video_0', 'video_0', 'video_0', 'video_0', '..."
2,user_29734049,"['video_0', 'video_0', 'video_0', 'video_0', '..."
3,user_955460,"['video_0', 'video_0', 'video_0', 'video_0', '..."
4,user_7065521,"['video_0', 'video_0', 'video_0', 'video_0', '..."
...,...,...
97235,user_29281681,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97236,user_3912848,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97237,user_28389099,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97238,user_18951296,"['video_0', 'video_0', 'video_0', 'video_0', '..."


In [5]:
player_starts.isna().sum()

date            0
user_id         0
item_id         0
watch_time      0
is_autorized    0
dtype: int64

In [6]:
player_starts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69954380 entries, 0 to 69954379
Data columns (total 5 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   date          object
 1   user_id       object
 2   item_id       object
 3   watch_time    int64 
 4   is_autorized  int64 
dtypes: int64(2), object(3)
memory usage: 2.6+ GB


Преобразуем числовые типы

In [7]:
for col in player_starts.select_dtypes(np.number).columns:
    player_starts[col] = pd.to_numeric(player_starts[col], downcast='integer')

In [8]:
player_starts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69954380 entries, 0 to 69954379
Data columns (total 5 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   date          object
 1   user_id       object
 2   item_id       object
 3   watch_time    int32 
 4   is_autorized  int8  
dtypes: int32(1), int8(1), object(3)
memory usage: 1.9+ GB


In [9]:
%%time
player_starts = player_starts.drop_duplicates(subset=['date', 'user_id', 'item_id'])
player_starts.shape

CPU times: user 54.9 s, sys: 2.73 s, total: 57.6 s
Wall time: 57.5 s


(69930877, 5)

Исключаем видео без watch_time

In [10]:
player_starts = player_starts[player_starts['watch_time']!=-1]

In [11]:
%%time
player_starts['date_short'] = player_starts['date'].apply(lambda l: l.split(" ")[0])
player_starts

CPU times: user 8.11 s, sys: 856 ms, total: 8.97 s
Wall time: 8.97 s


,date,user_id,item_id,watch_time,is_autorized,date_short
0,2023-07-21 19:04:50+03:00,user_12964323,video_1042531,51,0,2023-07-21
1,2023-07-21 02:02:41+03:00,user_16517,video_1707159,31,0,2023-07-21
2,2023-07-21 22:00:47+03:00,user_15057892,video_1989987,9,0,2023-07-21
4,2023-07-21 11:06:58+03:00,user_20517034,video_1380654,11,0,2023-07-21
5,2023-07-21 23:24:41+03:00,user_8293675,video_331810,340,0,2023-07-21
...,...,...,...,...,...,...
69954375,2023-08-21 02:51:53+03:00,user_15478739,video_1449287,291,1,2023-08-21
69954376,2023-08-21 08:40:18+03:00,user_25783543,video_1423321,2,0,2023-08-21
69954377,2023-08-21 05:19:55+03:00,user_3507470,video_464555,261,0,2023-08-21
69954378,2023-08-21 12:32:32+03:00,user_13128840,video_420973,21,0,2023-08-21


Формируем категориальные типы

In [12]:
%%time
from pandas.api.types import CategoricalDtype

users = set(pd.concat([player_starts['user_id'], sample_submission['user_id']]))
cat_users = CategoricalDtype(categories=users)
player_starts['user_id'] = player_starts['user_id'].astype(cat_users)
sample_submission['user_id'] = sample_submission['user_id'].astype(cat_users)

player_starts['item_id'] = player_starts['item_id'].astype('category')
items = player_starts['item_id'].cat.categories

CPU times: user 47.1 s, sys: 1.42 s, total: 48.6 s
Wall time: 48.5 s


In [13]:
index_user_id = dict(enumerate(cat_users.categories))
index_item_id = dict(enumerate(player_starts['item_id'].cat.categories))

In [14]:
%%time
count_user_date = player_starts.groupby(['user_id', 'date_short'])['item_id'].transform('count')

CPU times: user 13 s, sys: 1.05 s, total: 14 s
Wall time: 14 s


Оставляем пользователей с просмотрами за сессию, более или равным пороговому значению

In [15]:
N_ITEMS_SESSION = 2
player_starts['is_active'] = count_user_date >= N_ITEMS_SESSION

In [16]:
player_starts_active = player_starts[player_starts['is_active']]

Формируем разреженную матрицу user-item

In [17]:
%%time
player_starts_sparse = csr_matrix((np.ones(player_starts_active.shape[0]), 
                                (player_starts_active.user_id.cat.codes, 
                                 player_starts_active.item_id.cat.codes)), dtype='int8')

CPU times: user 1.48 s, sys: 52.1 ms, total: 1.53 s
Wall time: 1.53 s


In [18]:
%%time
model = TFIDFRecommender()
model.fit(player_starts_sparse)

  0%|          | 0/2245478 [00:00<?, ?it/s]

CPU times: user 3min 43s, sys: 3.77 s, total: 3min 47s
Wall time: 20.5 s


Получаем рекомендации

In [19]:
%%time
N_PREDICT = 100
N_TOP = 10
FILTER_ALREADY_LIKED_ITEMS = True

users_ = sample_submission['user_id'].cat.codes.unique()
ids, scores = model.recommend(users_, player_starts_sparse.tocsr()[users_].astype(float), N=N_PREDICT, 
                              filter_already_liked_items=FILTER_ALREADY_LIKED_ITEMS)

CPU times: user 5.45 s, sys: 19.5 ms, total: 5.47 s
Wall time: 5.47 s


In [20]:
%%time
ids_ = ids.flatten()
rec_df = pd.DataFrame([], index=range(ids_.shape[0]), columns=['user_id', 'item_id'])
rec_df['user_id'] = np.repeat(users_, N_PREDICT)
rec_df['user_id'] = rec_df['user_id'].map(index_user_id)
rec_df['item_id'] = ids_
rec_df['item_id'] = rec_df['item_id'].map(index_item_id)
rec_df = rec_df.drop_duplicates().dropna()
rec_df

CPU times: user 10.5 s, sys: 749 ms, total: 11.3 s
Wall time: 11.3 s


,user_id,item_id
0,user_26511551,video_1090606
1,user_26511551,video_474547
2,user_26511551,video_360688
3,user_26511551,video_1494923
4,user_26511551,video_1620648
...,...,...
9723939,user_16411220,video_727931
9723940,user_16411220,video_473473
9723941,user_16411220,video_1022134
9723942,user_16411220,video_1180057


Исключаем из рекоммендаций уже просмотренные пользователем видео

In [21]:
%%time
player_starts_active['user_id_item_id'] = list(zip(player_starts_active['user_id'], 
                                                   player_starts_active['item_id']))
rec_df['user_id_item_id'] = list(zip(rec_df['user_id'], rec_df['item_id']))
rec_df_ = rec_df[~rec_df['user_id_item_id'].isin(player_starts_active['user_id_item_id'])]

CPU times: user 20.4 s, sys: 1.13 s, total: 21.5 s
Wall time: 21.5 s


Некоторые пользователи остаются с менее, чем 10-тью рекомендациями или, вообще, без рекоммендаций

In [22]:
count_sample_users = len(set(sample_submission.user_id))
count_rec_users = len(set(rec_df_.user_id))
print('COUNT SAMPLE USERS =', count_sample_users)
print('COUNT REC USERS =', count_rec_users)
print('DIFF =', count_sample_users - count_rec_users)

COUNT SAMPLE USERS = 97240
COUNT REC USERS = 51777
DIFF = 45463


Рекомендуем этим пользователям самое популярное по количеству просмотров - случайным выбором

In [23]:
POPULAR_VIDEO = player_starts_active.item_id.value_counts()[:N_PREDICT].index
user_ids = sample_submission.user_id[~sample_submission.user_id.isin(rec_df_.user_id)]\
                                    .drop_duplicates().dropna().to_frame()
user_ids = pd.concat([user_ids, rec_df_.user_id.value_counts()[rec_df_.user_id.value_counts()<N_TOP]\
                      .reset_index().drop('user_id', axis=1).rename(columns={'index': 'user_id'})])                          
user_ids = pd.DataFrame(np.repeat(user_ids.values, N_PREDICT), columns=['user_id'])
user_ids['item_id'] = np.array([POPULAR_VIDEO] * (user_ids.shape[0] // N_PREDICT)).reshape(-1, 1)
user_ids = user_ids.sample(frac=1.0)
user_ids

,user_id,item_id
4093269,user_29477948,video_1366097
3681721,user_29245646,video_1628773
2537006,user_29461829,video_836422
727766,user_26061211,video_45935
3085676,user_29488001,video_1639963
...,...,...
4367080,user_29350744,video_2120061
2410495,user_29519030,video_1721228
1700262,user_29720366,video_766725
4425955,user_29495854,video_291331


Исключаем из реккомендаций уже просмотренные пользователем видео и оставляем N_TOP рекомендаций

In [24]:
rec_df_['user_id_item_id'] = list(zip(rec_df_['user_id'], rec_df_['item_id']))
rec_df_ = rec_df_[~rec_df_['user_id_item_id'].isin(player_starts_active['user_id_item_id'])]

In [25]:
rec_df_ = pd.concat([rec_df_[['user_id', 'item_id']], user_ids])
rec_df_ = rec_df_.drop_duplicates().dropna()
rec_df_ = rec_df_.groupby('user_id')[['user_id', 'item_id']].head(N_TOP)
rec_df_

,user_id,item_id
0,user_26511551,video_1090606
1,user_26511551,video_474547
2,user_26511551,video_360688
3,user_26511551,video_1494923
4,user_26511551,video_1620648
...,...,...
309739,user_29486330,video_2194155
309773,user_29486330,video_1631397
4361218,user_29929697,video_1989987
1310684,user_29209863,video_1643660


In [26]:
count_sample_users = len(set(sample_submission.user_id))
count_rec_users = len(set(rec_df_.user_id))
print('COUNT SAMPLE USERS =', count_sample_users)
print('COUNT REC USERS =', count_rec_users)
print('DIFF =', count_sample_users - count_rec_users)

COUNT SAMPLE USERS = 97240
COUNT REC USERS = 97240
DIFF = 0


In [27]:
rec_df_['user_id'].value_counts()

user_26511551    10
user_29584672    10
user_29544362    10
user_13265765    10
user_29234253    10
                 ..
user_6677785     10
user_24086582    10
user_13650551    10
user_954577      10
user_29495586    10
Name: user_id, Length: 97240, dtype: int64

# VIEW SUBMIT

In [28]:
%%time
features = pd.read_parquet("../input/videos.parquet").drop_duplicates(subset=['item_id'])

CPU times: user 7.86 s, sys: 3.47 s, total: 11.3 s
Wall time: 8.95 s


In [29]:
rec_ = rec_df_.merge(features, on=['item_id'], how='left').sort_values('user_id')

Уже просмотренное случайным пользователем

In [30]:
%%time

RANDOM_USER = rec_.loc[(rec_['user_id'].isin(set(sample_submission['user_id'])))\
                       &(rec_['user_id'].isin(set(player_starts_active['user_id']))), 'user_id']\
                                                    .sample(1).values[0]
player_starts[['user_id', 'item_id']].loc[player_starts['user_id'] == RANDOM_USER]\
                                                .merge(features, on=['item_id'], how='left')

CPU times: user 7.29 s, sys: 584 ms, total: 7.88 s
Wall time: 8.18 s


,user_id,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
0,user_22384443,video_1317715,"Звёзды в Африке, 2 сезон, 2 выпуск",Телеканал ТНТ,Новые звезды в Африке,2,Премьера! Новый сезон сверхпопулярного шоу о в...,Телепередачи,2022-03-07 11:00:18+03:00,4308200,75496,0,0.260896,0.269104,0.272408,0.269816
1,user_22384443,video_1317715,"Звёзды в Африке, 2 сезон, 2 выпуск",Телеканал ТНТ,Новые звезды в Африке,2,Премьера! Новый сезон сверхпопулярного шоу о в...,Телепередачи,2022-03-07 11:00:18+03:00,4308200,75496,0,0.260896,0.269104,0.272408,0.269816
2,user_22384443,video_1317715,"Звёзды в Африке, 2 сезон, 2 выпуск",Телеканал ТНТ,Новые звезды в Африке,2,Премьера! Новый сезон сверхпопулярного шоу о в...,Телепередачи,2022-03-07 11:00:18+03:00,4308200,75496,0,0.260896,0.269104,0.272408,0.269816
3,user_22384443,video_1462868,"Звёзды в Африке, 2 сезон, 1 выпуск",Телеканал ТНТ,Новые звезды в Африке,2,Премьера! Новый сезон сверхпопулярного шоу о в...,Телепередачи,2022-02-28 11:08:07+03:00,4624600,75496,0,0.047065,0.048948,0.052287,0.056341
4,user_22384443,video_1029886,"Звезды в Африке, 2 сезон, 3 выпуск",Телеканал ТНТ,Новые звезды в Африке,2,Премьера! Новый сезон сверхпопулярного шоу о п...,Телепередачи,2022-03-14 11:00:31+03:00,4188319,75496,0,0.126708,0.108296,0.112757,0.128120


Реккомендованное пользователю

In [31]:
rec_[rec_['user_id'] == RANDOM_USER]

,user_id,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
121605,user_22384443,video_2189663,"Звезды в Африке, 3 сезон, 3 выпуск",Телеканал ТНТ,Новые звезды в Африке,3,Премьера! В третьем сезоне экстремально-приклю...,Телепередачи,2022-09-26 11:00:25+03:00,5279280,75496,0,0.279689,0.252384,0.279100,0.278068
121606,user_22384443,video_1174960,"Звезды в Африке, 3 сезон, 4 выпуск",Телеканал ТНТ,Новые звезды в Африке,3,Премьера! В третьем сезоне экстремально-приклю...,Телепередачи,2022-10-03 11:00:55+03:00,5669400,75496,0,0.318654,0.322852,0.314294,0.312856
121607,user_22384443,video_2014114,"Звезды в Африке, 2 сезон, 14 выпуск",Телеканал ТНТ,Новые звезды в Африке,2,Премьера! Новый сезон сверхпопулярного шоу бли...,Телепередачи,2022-05-30 11:00:13+03:00,3889000,75496,0,0.229854,0.215304,0.237683,0.228554
121601,user_22384443,video_1101974,"Новые Звёзды в Африке, 13 выпуск",Телеканал ТНТ,Новые звезды в Африке,4,"Новые правила, новые участника, новые испытани...",Телепередачи,2023-06-05 18:55:12+03:00,5471000,75496,0,0.245847,0.222658,0.221927,0.249693
121604,user_22384443,video_769286,"Новые Звёзды в Африке, 14 выпуск",Телеканал ТНТ,Новые звезды в Африке,4,Полуфинал проекта «Новые звезды в Африке». В л...,Телепередачи,2023-06-11 21:00:12+03:00,5336160,75496,0,0.298426,0.246525,0.229877,0.224602
121602,user_22384443,video_1122625,"Новые Звёзды в Африке, 15 выпуск",Телеканал ТНТ,Новые звезды в Африке,4,Грандиозный финал проекта «Новые звезды в Афри...,Телепередачи,2023-06-18 21:00:13+03:00,5450840,75496,0,0.150769,0.128257,0.127336,0.115696
121610,user_22384443,video_170733,"Звезды в Африке, 2 сезон, 5 выпуск",Телеканал ТНТ,Новые звезды в Африке,2,Премьера! Новый сезон сверхпопулярного шоу о п...,Телепередачи,2022-03-28 11:00:31+03:00,4035960,75496,0,0.313193,0.267064,0.283326,0.264375
121609,user_22384443,video_665164,"Звезды в Африке, 3 сезон, 1 выпуск",Телеканал ТНТ,Новые звезды в Африке,3,Премьера! В третьем сезоне экстремально-приклю...,Телепередачи,2022-09-12 11:15:06+03:00,5543160,75496,0,0.160178,0.154598,0.164741,0.089296
121603,user_22384443,video_1903845,"Звезды в Африке, 3 сезон, 6 выпуск",Телеканал ТНТ,Новые звезды в Африке,3,Премьера! В третьем сезоне экстремально-приклю...,Телепередачи,2022-10-17 11:00:28+03:00,5618520,75496,0,0.301976,0.269007,0.298128,0.279062
121608,user_22384443,video_1090723,"Звезды в Африке, 3 сезон, 5 выпуск",Телеканал ТНТ,Новые звезды в Африке,3,Премьера! В третьем сезоне экстремально-приклю...,Телепередачи,2022-10-10 11:00:46+03:00,5902240,75496,0,0.308498,0.302654,0.317730,0.312130


# SUBMIT

In [32]:
rec_df_sub = rec_df_.groupby('user_id')['item_id'].apply(lambda x: list(x.values)).reset_index()

In [33]:
submission = sample_submission.drop('recs', axis=1).copy()
submission = submission.merge(rec_df_sub, on='user_id').rename(columns={'item_id': 'recs'})
submission

,user_id,recs
0,user_26511551,"[video_1090606, video_474547, video_360688, vi..."
1,user_29194819,"[video_814938, video_1761620, video_1009706, v..."
2,user_29734049,"[video_803844, video_836422, video_1764512, vi..."
3,user_955460,"[video_1738263, video_1086725, video_144691, v..."
4,user_7065521,"[video_2136380, video_1235290, video_598938, v..."
...,...,...
97235,user_29281681,"[video_845554, video_1075183, video_889838, vi..."
97236,user_3912848,"[video_315645, video_582126, video_433711, vid..."
97237,user_28389099,"[video_2210231, video_965653, video_1517964, v..."
97238,user_18951296,"[video_576750, video_993236, video_777075, vid..."


In [34]:
submission.to_csv("submission_.csv", index=False)

In [35]:
print("Время работы кода при непрерывном запуске, мин =", np.round((time() - start) / 60, 3))

Время работы кода при непрерывном запуске, мин = 4.747
